Primeiramente, a tabela foi visualizada e a identificação de dados desnecessários foi realizada, sendo um deles o ID do cliente. Logo em seguida, a linha com o método .drop e axis a coluna inteira foi removida.

In [ ]:
import pandas as pd

tabela = pd.read_csv("cancelamentos.csv")
tabela = tabela.drop("CustomerID", axis=1)
display(tabela)

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
881661,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
881662,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
881663,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
881664,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


Outra limpeza necessária é a remoção de valores 'NaN' (que representam valores desconhecidos).

In [ ]:
# identificando e removendo valores vazios
display(tabela.info())
tabela = tabela.dropna()
display(tabela.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 881666 entries, 0 to 881665
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idade                   881664 non-null  float64
 1   sexo                    881664 non-null  object 
 2   tempo_como_cliente      881663 non-null  float64
 3   frequencia_uso          881663 non-null  float64
 4   ligacoes_callcenter     881664 non-null  float64
 5   dias_atraso             881664 non-null  float64
 6   assinatura              881661 non-null  object 
 7   duracao_contrato        881663 non-null  object 
 8   total_gasto             881664 non-null  float64
 9   meses_ultima_interacao  881664 non-null  float64
 10  cancelou                881664 non-null  float64
dtypes: float64(8), object(3)
memory usage: 74.0+ MB


None

<class 'pandas.core.frame.DataFrame'>
Index: 881659 entries, 0 to 881665
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   idade                   881659 non-null  float64
 1   sexo                    881659 non-null  object 
 2   tempo_como_cliente      881659 non-null  float64
 3   frequencia_uso          881659 non-null  float64
 4   ligacoes_callcenter     881659 non-null  float64
 5   dias_atraso             881659 non-null  float64
 6   assinatura              881659 non-null  object 
 7   duracao_contrato        881659 non-null  object 
 8   total_gasto             881659 non-null  float64
 9   meses_ultima_interacao  881659 non-null  float64
 10  cancelou                881659 non-null  float64
dtypes: float64(8), object(3)
memory usage: 80.7+ MB


None

Após a limpeza, precisamos identificar o problema de cancelamento de assinaturas. Para isso, realizamos a contagem das pessoas que cancelaram (sendo 1 para quem cancelou e 0 para quem não cancelou). Para facilitar a visualização, esses números foram convertidos em porcentagem, proporcionando uma compreensão mais clara.

In [ ]:
# quantas pessoas cancelaram e não cancelaram
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

cancelou
1.0    499993
0.0    381666
Name: count, dtype: int64

cancelou
1.0    56.7%
0.0    43.3%
Name: proportion, dtype: object

Foi utilizado esse comando para verificar a igualdade entre os tipos de contrato, no entanto,  foi notado que há consideravelmente menos contratos mensais em comparação aos demais.

In [ ]:
display(tabela["duracao_contrato"].value_counts(normalize=True))
display(tabela["duracao_contrato"].value_counts())

duracao_contrato
Annual       0.401964
Quarterly    0.400448
Monthly      0.197588
Name: proportion, dtype: float64

duracao_contrato
Annual       354395
Quarterly    353059
Monthly      174205
Name: count, dtype: int64

Após a verificação, com base na duração do contrato, observamos que praticamente todos os contratos mensais foram cancelados.

In [ ]:
# analisando o contrato mensal
display(tabela.groupby("duracao_contrato").mean(numeric_only=True))
# descobrimos aqui que a média de cancelamentos é 1

,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
duracao_contrato,,,,,,,,
Annual,38.842165,31.446186,15.880213,3.263401,12.465156,651.697738,14.236107,0.460760
Monthly,41.552407,30.538555,15.499274,4.985649,15.007267,550.616435,15.478012,1.000000
Quarterly,38.830938,31.419916,15.886662,3.265245,12.460863,651.427783,14.234544,0.460255


Levando em consideração que os contratos mensais apresentam um problema, com todos sendo cancelados, podemos afirmar que isso é uma das causas dos 56% de cancelamentos. Portanto, vamos retirá-los da tabela e tentar identificar outras possíveis causas.

In [ ]:
# então descobrimos que contrato mensal é ruim, vamos tirar ele e continuar analisando
tabela = tabela[tabela["duracao_contrato"]!="Monthly"]
display(tabela)
display(tabela["cancelou"].value_counts())
display(tabela["cancelou"].value_counts(normalize=True).map("{:.1%}".format))

,idade,sexo,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,assinatura,duracao_contrato,total_gasto,meses_ultima_interacao,cancelou
0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
5,51.0,Male,33.0,25.0,9.0,26.0,Premium,Annual,129.00,8.0,1.0
6,58.0,Female,49.0,12.0,3.0,16.0,Standard,Quarterly,821.00,24.0,1.0
7,55.0,Female,37.0,8.0,4.0,15.0,Premium,Annual,445.00,30.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
881661,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
881662,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
881663,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
881664,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


cancelou
0.0    381666
1.0    325788
Name: count, dtype: int64

cancelou
0.0    53.9%
1.0    46.1%
Name: proportion, dtype: object

Após a remoção dos contratos mensais, a taxa de cancelamento caiu para 46%, uma redução de 10%. No entanto, ainda não atingimos uma taxa satisfatória de cancelamento, indicando a necessidade de uma análise mais abrangente.
A próxima análise será focada no tipo de assinatura.

In [ ]:
display(tabela["assinatura"].value_counts(normalize=True))
display(tabela.groupby("assinatura").mean(numeric_only=True))

assinatura
Standard    0.339648
Premium     0.338138
Basic       0.322215
Name: proportion, dtype: float64

,idade,tempo_como_cliente,frequencia_uso,ligacoes_callcenter,dias_atraso,total_gasto,meses_ultima_interacao,cancelou
assinatura,,,,,,,,
Basic,38.904813,32.316031,15.876921,3.310021,12.507054,648.642614,14.240814,0.475188
Premium,38.817814,30.977869,15.889673,3.235886,12.433427,653.337633,14.231150,0.452338
Standard,38.790478,31.048621,15.883393,3.249275,12.450690,652.566793,14.234280,0.454714
